In [1]:
import sys
sys.path.append('../../build-cdfpp-Desktop-debug/')
import pycdfpp
import pandas as pds
from astropy.time import Time
from datetime import timedelta,datetime
import numpy as np
from spacepy import pycdf
import cppyy
import cppyy.ll
cppyy.ll.set_signals_as_exception(True)

False

In [2]:
def seconds_since_0_AD_no_leap(date_str):
    date = Time(date_str)
    if date_str[0:4] == "0000":
        return (date - Time('0000-01-01 00:00:00', scale='tai')).sec
    else:
        dt = (date - Time('0000-01-01 00:00:00', scale='tai')).datetime
        return dt.days * 86400 + date.datetime.hour * 3600 + date.datetime.minute * 60 + date.datetime.second


def seconds_since_J2000(date_str):
    date = Time(date_str, scale='utc')
    return (date - Time('2000-01-01 11:59:27.816', scale='tai')).sec


def split_date(date: datetime):
    return date.year, date.month, date.day, date.hour, date.minute, date.second


#leap_seconds = pds.read_csv("https://www.ietf.org/timezones/data/leap-seconds.list", comment="#", sep='\t', names=["epoch","offset"], index_col=False)

In [3]:
cppyy.add_include_path('../include/')
cppyy.add_include_path('../date/include/')
cppyy.add_include_path('../../build-cdfpp-Desktop-debug/')
cppyy.add_include_path('/usr/local/cdf/include')
cppyy.add_library_path('/usr/local/cdf/lib')
cppyy.load_library('libcdf.so')

True

In [4]:
cppyy.cppdef(R'''
#include <chrono>
#include <ctime>
#include "cdf-chrono.hpp"
#include "date/date.h"
#include <cdf.h>

using namespace date;
using namespace std::chrono;
using namespace cdf;

std::array dates = { 
    //sys_days { 0000_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 500_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1500_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1800_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1900_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1960_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1961_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1962_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1968_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1970_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1986_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1996_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 1996_y / October / 23 } + 8h + 19min + 3s,
    sys_days { 1998_y / October / 23 } + 8h + 19min + 3s,
    sys_days { 2000_y / January / 1 } + 12h + 0min + 0s,
    sys_days { 2006_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 2010_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 2020_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 2000_y / January / 1 } + 0h + 0min + 0s,
    sys_days { 2020_y / July / 25 } + 2h + 35min + 8s };
    
using date_t = decltype(*std::begin(dates));
template<typename T>
auto str(const T& date){
    return format("%FT%T", floor<seconds>(date));
}

long long to_tt2000(double year=2000.,double month=1.,double day=1.,double hour=12.,double minute=0.,double seconde=0.)
{
    return computeTT2000(year,month,day,hour,minute,seconde,TT2000END);
}

std::pair<double,double> to_epoch16(long year=2000,long month=1,long day=1,long hour=12,
                                    long minute=0,long second=0, long msec=0, long microsec=0,
                                    long nanosec=0, long picosec=0
                                    )
{
    double epoch16[2];
    computeEPOCH16(year,month,day,hour,minute,second,msec,microsec,nanosec,picosec,epoch16);
    return {epoch16[0],epoch16[1]};
}
    
''')

from cppyy.gbl import cdf

In [5]:
datetime.fromisoformat(cppyy.gbl.str(cppyy.gbl.dates[9]))

datetime.datetime(1970, 1, 1, 0, 0)

In [6]:
start = 9
epochs = np.array([cdf.to_epoch(d).value for d in cppyy.gbl.dates])
epochs16 = np.array([cdf.to_epoch16(d).seconds for d in cppyy.gbl.dates])
tt2000s = np.array([
    cdf.to_tt2000(d).value / (1000 * 1000 * 1000) for d in cppyy.gbl.dates
])[start:]

ref_tt2000s = np.array(
    [seconds_since_J2000(cppyy.gbl.str(d)) for d in cppyy.gbl.dates])[start:]
cdf_tt2000s = np.array([
    cppyy.gbl.to_tt2000(*split_date(datetime.fromisoformat(cppyy.gbl.str(d))))
    for d in cppyy.gbl.dates
])[start:] / 1000000000

In [7]:
ref_tt2000s - tt2000s, cdf_tt2000s - tt2000s, ref_tt2000s - cdf_tt2000s

(array([-9.99917984e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.57740487e-12,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 array([-0.99862194,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]),
 array([-1.29604340e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.57740487e-12,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]))

In [8]:
d=cppyy.gbl.dates[11]
d2=cdf.to_time_point(cdf.to_tt2000(d))
cppyy.gbl.str(d),cppyy.gbl.str(d2)

('1996-01-01T00:00:00', '1996-01-01T00:00:00')

In [8]:
d=cppyy.gbl.dates[13]
date = datetime.fromisoformat(cppyy.gbl.str(d))
print(date)
pycdf_tt2000 = pycdf.lib.datetime_to_tt2000(date)/1000000000
astropy_tt2000 = seconds_since_J2000(str(date))
my_tt2000 = cdf.to_tt2000(d).value/1000000000

2000-01-01 12:00:00


In [13]:
seconds_since_J2000(datetime(2000,1,1,12))

64.18399999999842

In [15]:
seconds_since_J2000(datetime(1996,1,1,0,0))

-126273537.816

In [10]:
pycdf_tt2000,my_tt2000

(64.184, 64.184)

In [11]:
my_tt2000 - pycdf_tt2000

0.0

In [16]:
cppyy.gbl.to_tt2000(2000.,1.,1.)/1000000000

64.184

In [15]:
cppyy.gbl.to_tt2000(2000.,1.,1.,11.,59.,59.)/1000000000

63.184

In [16]:
cppyy.gbl.to_tt2000(2000.,1.,1.,11.,59.,59.)/1000000000

63.184

In [17]:
cppyy.gbl.to_epoch16(2020,1,1,12,0,0).first - cppyy.gbl.to_epoch16(2000,1,1,12,0,0).first

631152000.0

In [18]:
cppyy.gbl.to_tt2000(2020.,1.,1.,12.,0.,0.)/1000000000

631152069.184

In [19]:
cppyy.gbl.to_tt2000(2019.,12.,31.,0.,0.,0.)/1000000000

631022469.184

In [14]:
datetime(2000,1,1,12,0,0,0)-datetime(2000,1,1,11,58,55,816000)

datetime.timedelta(seconds=64, microseconds=184000)